## Improvements
* Cannot use Tor
* Reindex (There are duplicate key indexes)

In [1]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np
import math

In [ ]:
## Steps for Git
Git add <file> 
Git commit -m "comments" 
Git push origin master

In [21]:
## Initizlization

#Create SQLite
conn = sqlite3.connect('songlyrics.db')
df = pd.read_sql_query("select * from songlyrics_url;",conn)

In [37]:
user_agent_list = ['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
                   'Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0'
                  ]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


session = requests.session()
session.proxies = {'https':'socks5://127.0.0.1:9050','http':'socks5://127.0.0.1:9050'}

## Get Christian Artist List

In [40]:
artist_list = pd.read_csv('christian_artist.csv')

In [22]:
list_url = 
['http://www.songlyrics.com/twila-paris-lyrics/',
 'http://www.songlyrics.com/wayne-watson-lyrics/'
]

'don moen'

In [75]:
def clean_artist(x):
#     artist_list[artist_list['Genre']=='Band'].iloc[20,0].split('\xa0')[0]
    return x['Artist'].split('\xa0')[0]

artist_list['clean_artist'] = artist_list.apply(lambda x: clean_artist(x),axis=1)

In [77]:
def convert_url(x):
    return "http://www.songlyrics.com/"+x['clean_artist'].lower().replace(',','').replace('.','').replace(' &','').replace(' ','-')+"-lyrics/"

artist_list['url'] = artist_list.apply(lambda x: convert_url(x),axis=1)

In [86]:
artist_list[artist_list.duplicated(subset=['clean_artist'])]
final_list = artist_list.drop_duplicates(subset=['clean_artist'],inplace=True)
# artist_list[artist_list['clean_artist']=='Newsboys']
artist_list.to_csv('christian_artist_clean.csv')

In [99]:
# 0 not yet
# 1 downloaded
# 2 artist name don't match
artist_list['status'] = 0

In [125]:
artist_list['Genre'].unique()

array(['Alternative_Rock', 'Band', 'Contemporary_Pop', 'Country',
       'Country_', 'Gospel', 'Gospel_', 'Inspirational', 'Modern_Rock',
       'Praise and Worship', 'Vocal', 'Worship'], dtype=object)

In [133]:
url_list = artist_list[(artist_list['Genre']=='Worship')|
                       (artist_list['Genre']=='Gospel')|(artist_list['Genre']=='Gospel_')|
                       (artist_list['Genre']=='Inspirational')|
                       (artist_list['Genre']=='Contemporary_Pop')|
                       (artist_list['Genre']=='Praise and Worship')|
                       (artist_list['Genre']=='Country')|(artist_list['Genre']=='Country_')|
                       (artist_list['Genre']=='Vocal')]

In [132]:
len(url_list)

568

In [134]:
base_url = "http://www.songlyrics.com/steve-green-lyrics/"

columns = ['url','title','status','artist']
dflyrics = pd.DataFrame(columns=columns)

for key,row in url_list.iterrows():
    # response = session.get(url, headers=headers)
    pause = randint(3,5)
    time.sleep(pause)
    base_url = row['url']
    response = requests.get(base_url,headers=headers)
    response_url = ' '.join(response.url.split("/")[-2].split("-")[0:-1])
    artist = base_url.split("/")[-2].replace('lyrics','').replace('-',' ').strip().lower()
    
    
    if response_url != artist:
        url_list.loc[key,'status'] = 2
    else:
        url_list.loc[key,'status'] = 1
        data = response.text
        song_url = BeautifulSoup(data,"lxml")
        song_url_list = song_url.findAll('table',attrs={'class','tracklist'})
        for s in song_url_list[0].findAll('a'):
            url = s.get('href')
            title = s.get_text()
            status = 0 
        #     artist = base_url.split("/")[-2].replace('lyrics','').replace('-',' ').strip()
            row = [url,title,status,artist]
            dflyrics.loc[len(dflyrics)] = row  

dflyrics.to_sql('songlyrics_url',conn,if_exists='append')

/home/chenjennhaur/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [138]:
url_list[url_list['status']==2].to_csv('error_url.csv')

In [139]:
songlyrics = pd.read_sql_query('select * from songlyrics_url',conn)
len(songlyrics)

41066

In [148]:
# songlyrics.drop_duplicates(subset=['url'],inplace=True)
songlyrics.to_sql('songlyrics_url',conn,if_exists='replace')

In [151]:

songlyrics.reset_index(inplace=True)
songlyrics['key'] = songlyrics['index']
del songlyrics['index']
#alter table songlyrics_url add column fullhtml text;

songlyrics.to_sql('songlyrics_url',conn,if_exists='replace')

## Get Lyrics

In [44]:
def set_new_ip():
    """Change IP using TOR"""
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)


def scraplyrics(initial_count,minibatch,lower_limit,upper_limit,headers,user_agent_list,conn):
    iterations = 0
    count_query = pd.read_sql_query("select count(*) as cnt from songlyrics_url where status = 0;",conn)
    count = int(count_query['cnt'][0])
    cursor = conn.cursor()
    while count > initial_count : 

        count_query = pd.read_sql_query("select count(*) as cnt from songlyrics_url where status = 0;",conn)
        count = int(count_query['cnt'][0])


        # Randomize 
        set_new_ip()
        rand_user_agent = randint(0,len(user_agent_list)-1)
        headers['User-Agent'] = user_agent_list[rand_user_agent]
        elapsed_time = 0 
        # Scrap
        scrap = pd.read_sql_query("select key,url from songlyrics_url where status = 0 limit "+str(mini_batch)+";",conn)
        t0 = time.time()
        for key,row in scrap.iterrows():

            pause = randint(lower_limit,upper_limit)
            time.sleep(pause)

            key = row['key']
            url = row['url']
            print(url)
    #         response = requests.get(url, headers=headers)
            response = session.get(url, headers=headers)
            print(session.headers)
            print(response.headers)
            elapsed_time += response.elapsed.total_seconds()
            data = response.text
            if 'CAPTCHA' in data :
                print("Blocked!")       
                return

            cursor.execute("""UPDATE songlyrics_url SET fullhtml = ?, status = ? WHERE key = ? """,
                  (data,1,key))
            conn.commit()

        t1 = time.time()


        print("Interations :",iterations," Per Request:" ,elapsed_time/mini_batch," Total:",t1-t0)
        # print("Initial_count ",initial_count," Current:",count)
        iterations = iterations + 1


    cursor.close()
    # conn.close()

        


# initial_count = count - batch

scraplyrics(37830,3,5,8,headers,user_agent_list,conn)

http://www.songlyrics.com/don-moen/the-greatness-of-you-lyrics/
{'User-Agent': 'python-requests/2.13.0', 'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*'}
{'Content-Type': 'text/html; charset=UTF-8', 'Connection': 'close', 'Transfer-Encoding': 'chunked', 'Expires': 'Mon, 29 May 2017 13:36:19 GMT', 'CF-RAY': '3669d77d46f7693e-CDG', 'Date': 'Mon, 29 May 2017 13:36:17 GMT', 'Server': 'cloudflare-nginx', 'Content-Encoding': 'gzip', 'Cache-Control': 'max-age=2', 'X-Frame-Options': 'SAMEORIGIN'}
Blocked!


In [ ]:
# Maranatha Music / Integrity Music / Vineyard Music / Passion

In [187]:
df = pd.read_sql_query("select * from songlyrics_url where status = 1",conn)
len(df)

60

In [191]:
df

,index,level_0,url,title,status,artist,key,fullhtml
0,0,0,http://www.songlyrics.com/don-moen/the-greatne...,The Greatness Of You,1.0,don moen,0,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
1,1,1,http://www.songlyrics.com/don-moen/o-god-of-ab...,O God Of Abraham,1.0,don moen,1,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
2,2,2,http://www.songlyrics.com/don-moen/grace-is-en...,Grace Is Enough,1.0,don moen,2,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
3,3,3,http://www.songlyrics.com/don-moen/saviour-kin...,Saviour King,1.0,don moen,3,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
4,4,4,http://www.songlyrics.com/don-moen/thank-you-l...,Thank You Lord,1.0,don moen,4,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
5,5,5,http://www.songlyrics.com/don-moen/i-will-sing...,I Will Sing,1.0,don moen,5,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
6,259,277,http://www.songlyrics.com/maranatha!/how-deep-...,How Deep the Father's Love for Us,1.0,maranatha!,0,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
7,260,278,http://www.songlyrics.com/maranatha!/amazing-g...,Amazing Grace,1.0,maranatha!,1,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
8,261,279,http://www.songlyrics.com/maranatha!/how-great...,How Great Thou Art,1.0,maranatha!,2,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."
9,262,280,http://www.songlyrics.com/maranatha!/all-hail-...,All Hail The Power Of Jesus' Name,1.0,maranatha!,3,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."


In [188]:
mask = df['fullhtml'].str.contains("CAPTCHA")
itemindex = np.where(mask==True)
print(itemindex)
# df['fullhtml'][468]

(array([], dtype=int64),)


In [189]:
#Page Extract

# base_url = "http://www.songlyrics.com/maranatha!-music/healer-lyrics/"
# response = requests.get(base_url)
# data = response.text

soup = BeautifulSoup(df['fullhtml'][0],"lxml")


In [190]:
df['fullhtml'][10]

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="https://www.w3.org/1999/xhtml" lang="en-US">\n<head>\n\t<title>DON MOEN - THANK YOU LORD LYRICS</title>\n\t<meta http-equiv="content-type" content="text/html; charset=utf-8" />\n\t<meta name="description" content="Don Moen - Thank You Lord Lyrics. Bible says it\'s a good thing to give thanks to the Lord, amen I come before You today And there\'s just one thing that I want to say Thank Y" />\n\t\n\t<meta name="msvalidate.01" content="CF28C9C2E5FDBD5C7CA7F6FE394BD121" />\n        \t<meta name="robots" content="noydir, noodp" />\n        \t<meta property="fb:admins" content="31113169,100000933112467,1383859676" />\n\t<meta name="fb:app_id" content="1418134321780018" />\n\t<meta name="viewport" content="width=device-width, initial-scale=1"> <!-- viewport -->\n\t\n\t<meta property="og:title" content="Thank You Lord - Don Moen"/>\n\t<meta property="og:type

In [192]:
lyrics = soup.find('p',attrs={'id':'songLyricsDiv'})
lyrics.get_text()

"I wanna sing of Your love \nWanna sing of Your mercy \nI wanna tell the whole world \nOf the greatness of You \nSo I'll sing of Your love \nAnd I'll sing of Your mercy \nAnd I'll tell the whole world \nOf the greatness of You \n\nJesus, we lift up Your name \nJesus, we lift up Your name \nJesus, we lift up Your name \nJesus, we lift up Your name \n\nI wanna sing of Your Love (I wanna sing of Your Love) \nI wanna sing of Your mercy (I wanna sing of Your mercy) \nI wanna tell the whole world (I wanna tell the whole world) \nOf the greatness of You (Of the greatness of You) \nSo I'll sing of Your love (So I'll sing of Your love) \nAnd I'll sing of Your mercy (And I'll sing of Your mercy) \nAnd I'll tell the whole world (And I'll tell the whole world) \nOf the greatness of You (Of the greatness of You) \n\nJesus, we lift up Your name \nJesus, we lift up Your name \nJesus, we lift up Your name \nJesus, we lift up Your name \n\nBackground: \nLord, we lift You up \n(Lord, we have come to lif